In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

np.random.RandomState(17)
random.seed(17)

In [ ]:
fca = sc.read(r".\raw_FCA_annotated.h5ad")
fca

AnnData object with n_obs × n_vars = 31401 × 12460
    obs: 'batch', 'celda_decontx__clusters', 'celda_decontx__contamination', 'celda_decontx__doublemad_predicted_outliers', 'fca_id', 'id', 'log_n_counts', 'log_n_genes', 'n_counts', 'n_genes', 'percent_mito', 'sample_id', 'scrublet__doublet_scores', 'leiden_res0.4', 'leiden_res0.6', 'leiden_res0.8', 'leiden_res1.0', 'leiden_res1.2', 'leiden_res1.4', 'leiden_res1.6', 'leiden_res1.8', 'leiden_res10.0', 'leiden_res2.0', 'leiden_res4.0', 'leiden_res6.0', 'leiden_res8.0', 'annotation', 'broad_annotation', 'broad_annotation_extrapolated', 'label'
    var: 'name'
    obsm: 'X_x_pca', 'X_x_tsne', 'X_x_umap'

In [ ]:
def clean_label(label):
    label = str(label).strip()
    
    if "??" in label or "?" in label:
        label = label.replace("??", "").replace("?", "").strip()
        label += " (uncertain)"
    
    label = label.replace("FCA_", "").replace("CSantos_", "")
    
    return label

fca_soma = fca[fca.obs['label'].str.startswith('CSantos')].copy()

fca_soma.obs['annotation'] = fca_soma.obs['label'].apply(clean_label)
fca_soma

AnnData object with n_obs × n_vars = 18571 × 12460
    obs: 'batch', 'celda_decontx__clusters', 'celda_decontx__contamination', 'celda_decontx__doublemad_predicted_outliers', 'fca_id', 'id', 'log_n_counts', 'log_n_genes', 'n_counts', 'n_genes', 'percent_mito', 'sample_id', 'scrublet__doublet_scores', 'leiden_res0.4', 'leiden_res0.6', 'leiden_res0.8', 'leiden_res1.0', 'leiden_res1.2', 'leiden_res1.4', 'leiden_res1.6', 'leiden_res1.8', 'leiden_res10.0', 'leiden_res2.0', 'leiden_res4.0', 'leiden_res6.0', 'leiden_res8.0', 'annotation', 'broad_annotation', 'broad_annotation_extrapolated', 'label'
    var: 'name'
    obsm: 'X_x_pca', 'X_x_tsne', 'X_x_umap'

In [4]:
fca_soma.obsm["X_pca"] = fca_soma.obsm.pop("X_x_pca")
fca_soma.obsm["X_tsne"] = fca_soma.obsm.pop("X_x_tsne")
fca_soma.obsm["X_umap"] = fca_soma.obsm.pop("X_x_umap")

fca_soma


AnnData object with n_obs × n_vars = 18571 × 12460
    obs: 'batch', 'celda_decontx__clusters', 'celda_decontx__contamination', 'celda_decontx__doublemad_predicted_outliers', 'fca_id', 'id', 'log_n_counts', 'log_n_genes', 'n_counts', 'n_genes', 'percent_mito', 'sample_id', 'scrublet__doublet_scores', 'leiden_res0.4', 'leiden_res0.6', 'leiden_res0.8', 'leiden_res1.0', 'leiden_res1.2', 'leiden_res1.4', 'leiden_res1.6', 'leiden_res1.8', 'leiden_res10.0', 'leiden_res2.0', 'leiden_res4.0', 'leiden_res6.0', 'leiden_res8.0', 'annotation', 'broad_annotation', 'broad_annotation_extrapolated', 'label'
    var: 'name'
    obsm: 'X_pca', 'X_tsne', 'X_umap'

In [ ]:
fca_soma.obs["annotation"] = fca_soma.obs["annotation"].cat.add_categories([
    "MB St1-6-like cells (uncertain identity)"
])

fca_soma.obs["annotation"] = fca_soma.obs["annotation"].mask(
    fca_soma.obs["annotation"].str.contains("MB St1-6") & 
    fca_soma.obs["annotation"].str.contains("don't think it's MB"),
    "MB St1-6-like cells (uncertain identity)"
)
fca_soma.obs["annotation"] = fca_soma.obs["annotation"].cat.remove_unused_categories()

In [6]:
fca_soma.layers["counts"] = fca_soma.X.copy()
oviductless = fca_soma[~fca_soma.obs['label'].str.contains("oviduct", case=False, na=False), :]


sc.pp.log1p(oviductless)
sc.pp.highly_variable_genes(oviductless, layer="counts", n_top_genes=2000, flavor="seurat_v3")

sc.tl.pca(oviductless, n_comps=50)
sc.pp.neighbors(oviductless)

c:\Users\unite\miniconda3\envs\label_harmony\Lib\site-packages\scanpy\preprocessing\_simple.py:404: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
c:\Users\unite\miniconda3\envs\label_harmony\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
sc.tl.umap(oviductless)
sc.pl.umap(oviductless, color=["annotation"], ncols = 1)

## Gene sets

In [ ]:
from bioservices import KEGG
import pandas as pd

kegg = KEGG()
kegg.organism = "dme"

pathway_list = kegg.list("pathway", organism="dme").strip().split("\n")

pathways_df = pd.DataFrame([line.split("\t") for line in pathway_list], 
                           columns=["pathway_id", "description"])

pathways_of_interest = [
    "00010",  # Glycolysis / Gluconeogenesis
    "00020",  # Citrate cycle (TCA cycle)
    "00030",  # Pentose phosphate pathway
    "00040",  # Pentose and glucuronate interconversions
    "00051",  # Fructose and mannose metabolism
    "00052",  # Galactose metabolism
    "00053",  # Ascorbate and aldarate metabolism
    "00500",  # Starch and sucrose metabolism
    "00620",  # Pyruvate metabolism
    "00630",  # Glyoxylate and dicarboxylate metabolism
    "00640",  # Propanoate metabolism
    "00650",  # Butanoate metabolism
    "00562",  # Inositol phosphate metabolism
    "00190",  # Oxidative phosphorylation
    "00680",  # Methane metabolism
    "00910",  # Nitrogen metabolism
    "00061",  # Fatty acid biosynthesis
    "00062",  # Fatty acid elongation
    "00071",  # Fatty acid degradation
    "00140",  # Steroid biosynthesis
    "00561",  # Glycerolipid metabolism
    "00564",  # Glycerophospholipid metabolism
    "00565",  # Sphingolipid metabolism
    "00600",  # Selenocompound metabolism
    "00590",  # Arachidonic acid metabolism
    "00591",  # Linoleic acid metabolism
    "00592",  # Alpha-linolenic acid metabolism
    "00230",  # Purine metabolism
    "00240",  # Pyrimidine metabolism
    "00250",  # Amino sugar and nucleotide sugar metabolism
    "00260",  # Glycosaminoglycan degradation
    "00270",  # One carbon pool by folate
    "00280",  # Valine, leucine and isoleucine degradation
    "00290",  # Valine, leucine and isoleucine biosynthesis
    "00310",  # Lysine biosynthesis
    "00220",  # Arginine biosynthesis
    "00330",  # Arginine and proline metabolism
    "00340",  # Histidine metabolism
    "00350",  # Tyrosine metabolism
    "00360",  # Phenylalanine metabolism
    "00380",  # Tryptophan metabolism
    "00400",  # Phenylpropanoid biosynthesis
    "00410",  # Beta-alanine metabolism
    "00430",  # Propanoate metabolism (alternative)
    "00480",  # Steroid hormone biosynthesis
    "00520",  # Amino sugar and nucleotide sugar metabolism (alternative)
    "00510",  # N-Glycan biosynthesis
    "00512",  # N-Glycan biosynthesis (variant)
    "00513",  # N-Glycan biosynthesis (alternative)
    "01040",  # Biosynthesis of unsaturated fatty acids
    "00514",  # Other glycan degradation
    "00515",  # Glycosphingolipid biosynthesis – ganglio series
    "00563",  # Glycosaminoglycan degradation (alternative)
    "00601",  # Glycosphingolipid biosynthesis – globo series
    "00603",  # Glycosphingolipid biosynthesis – lacto and neolacto series
    "00604",  # Glycosphingolipid biosynthesis – ganglio series (variant)
    "00730",  # Thiamine metabolism
    "00740",  # Riboflavin metabolism
    "00750",  # Pantothenate and CoA biosynthesis
    "00760",  # Biotin metabolism
    "00770",  # Folate biosynthesis
    "00780",  # Vitamin B6 metabolism
    "00785",  # Nicotinate and nicotinamide metabolism
    "00790",  # Lipoic acid metabolism
    "00830",  # Retinol metabolism
    "00670",  # One carbon pool by folate (duplicate entry, may require review)
    "00130",  # Ubiquinone and other terpenoid-quinone biosynthesis
    "00900",  # Terpenoid backbone biosynthesis
    "00981"   # Secondary bile acid biosynthesis
]


filtered_df = pathways_df[pathways_df["pathway_id"].str[-5:].isin(pathways_of_interest)]

gene_sets = {}

for idx, row in filtered_df.iterrows():
    pathway_id = row['pathway_id'].replace('path:', '')
    pathway_name = row['description']
    genes_info = kegg.get(pathway_id)
    parsed = kegg.parse(genes_info)
    
    genes = []
    if 'GENE' in parsed:
        genes = [gene_id for gene_id in parsed['GENE'].keys()]
    
    gene_sets[pathway_name] = genes

kegg_gene_sets_df = pd.DataFrame(
    [(pathway, gene) for pathway, genes in gene_sets.items() for gene in genes],
    columns=["pathway", "gene_symbol"]
)

print(kegg_gene_sets_df.head())


WARNING [bioservices.KEGG:130]:  The URL (http://rest.kegg.jp) provided cannot be reached.


                                             pathway   gene_symbol
0  Glycolysis / Gluconeogenesis - Drosophila mela...  Dmel_CG10160
1  Glycolysis / Gluconeogenesis - Drosophila mela...  Dmel_CG10202
2  Glycolysis / Gluconeogenesis - Drosophila mela...  Dmel_CG10467
3  Glycolysis / Gluconeogenesis - Drosophila mela...  Dmel_CG10924
4  Glycolysis / Gluconeogenesis - Drosophila mela...  Dmel_CG10996


In [ ]:
net_raw = kegg_gene_sets_df.rename(columns={"pathway": "source",
                                                 "gene_symbol": "target"})
net_raw['target'] = net_raw['target'].str.replace("Dmel_", "", regex=False)




net_mapped   = net_raw.copy()                         
net_filtered = None                                   

In [ ]:
###mapping is performed here with the keys being extracted with the help of https://www.biotools.fr/drosophila/fbgn_converter 
mapping_df   = pd.read_csv(r"id_validation_table_92436.txt",
                           sep="\t")
mapping_dict = mapping_df.set_index("#submitted_item")["current_symbol"].to_dict()

dataset_genes = set(oviductless.var_names)

net_mapped['cg_id']  = net_mapped['target']
net_mapped['symbol'] = net_mapped['cg_id'].map(mapping_dict)

def choose_id(row):
    sym, cg = row['symbol'], row['cg_id']
    if pd.notna(sym) and sym in dataset_genes:
        return sym
    if cg in dataset_genes:
        return cg
    return pd.NA                             

net_mapped['target'] = net_mapped.apply(choose_id, axis=1)

In [47]:
net_filtered = (
    net_mapped.dropna(subset=['target'])
              .groupby('source')
              .filter(lambda g: 10 <= g['target'].nunique() <= 500)
              .copy()
)

In [48]:
orig_size = (net_raw.groupby('source')['target']
                   .nunique()
                   .rename('orig_size'))

post_size = (net_filtered.groupby('source')['target']
                      .nunique()
                      .rename('post_size'))

stats = (pd.concat([orig_size, post_size], axis=1)
           .fillna(0)
           .assign(lost=lambda d: d.orig_size - d.post_size,
                   pct_retained=lambda d: 100 * d.post_size / d.orig_size)
           .astype({'orig_size': int, 'post_size': int, 'lost': int})
           .sort_values('pct_retained'))

print("TOP 20 most affected pathways:")
display(stats.head(20))

print("\nTOP 10 best-covered pathways:")
display(stats.sort_values('pct_retained', ascending=False).head(10))

TOP 20 most affected pathways:


,orig_size,post_size,lost,pct_retained
source,,,,
Vitamin B6 metabolism - Drosophila melanogaster (fruit fly),6,0,6,0.000000
Glycosphingolipid biosynthesis - lacto and neolacto series - Drosophila melanogaster (fruit fly),3,0,3,0.000000
Phenylalanine metabolism - Drosophila melanogaster (fruit fly),8,0,8,0.000000
"Phenylalanine, tyrosine and tryptophan biosynthesis - Drosophila melanogaster (fruit fly)",4,0,4,0.000000
Mannose type O-glycan biosynthesis - Drosophila melanogaster (fruit fly),4,0,4,0.000000
Taurine and hypotaurine metabolism - Drosophila melanogaster (fruit fly),9,0,9,0.000000
Glycosphingolipid biosynthesis - ganglio series - Drosophila melanogaster (fruit fly),5,0,5,0.000000
Histidine metabolism - Drosophila melanogaster (fruit fly),8,0,8,0.000000
"Valine, leucine and isoleucine biosynthesis - Drosophila melanogaster (fruit fly)",2,0,2,0.000000



TOP 10 best-covered pathways:


,orig_size,post_size,lost,pct_retained
source,,,,
Fatty acid biosynthesis - Drosophila melanogaster (fruit fly),13,13,0,100.000000
Nicotinate and nicotinamide metabolism - Drosophila melanogaster (fruit fly),19,18,1,94.736842
alpha-Linolenic acid metabolism - Drosophila melanogaster (fruit fly),13,12,1,92.307692
Lysine degradation - Drosophila melanogaster (fruit fly),35,32,3,91.428571
Inositol phosphate metabolism - Drosophila melanogaster (fruit fly),45,41,4,91.111111
Fatty acid degradation - Drosophila melanogaster (fruit fly),33,30,3,90.909091
Sphingolipid metabolism - Drosophila melanogaster (fruit fly),32,29,3,90.625000
Pyrimidine metabolism - Drosophila melanogaster (fruit fly),38,34,4,89.473684
Glycerophospholipid metabolism - Drosophila melanogaster (fruit fly),64,57,7,89.062500


In [49]:
print("Net shape:", net_filtered .shape)
print("Unique sources:", net_filtered ['source'].nunique())
print("Example net:\n", net_filtered .head())
print("Net shape:", net_raw.shape)
print("Unique sources:", net_raw['source'].nunique())
print("Example net:\n", net_raw.head())


Net shape: (1451, 4)
Unique sources: 53
Example net:
                                               source    target    cg_id  \
0  Glycolysis / Gluconeogenesis - Drosophila mela...       Ldh  CG10160   
2  Glycolysis / Gluconeogenesis - Drosophila mela...   CG10467  CG10467   
3  Glycolysis / Gluconeogenesis - Drosophila mela...    Pepck2  CG10924   
4  Glycolysis / Gluconeogenesis - Drosophila mela...   CG10996  CG10996   
5  Glycolysis / Gluconeogenesis - Drosophila mela...  Aldh-III  CG11140   

     symbol  
0       Ldh  
2     Galm2  
3    Pepck2  
4   CG10996  
5  Aldh-III  
Net shape: (1932, 2)
Unique sources: 66
Example net:
                                               source   target
0  Glycolysis / Gluconeogenesis - Drosophila mela...  CG10160
1  Glycolysis / Gluconeogenesis - Drosophila mela...  CG10202
2  Glycolysis / Gluconeogenesis - Drosophila mela...  CG10467
3  Glycolysis / Gluconeogenesis - Drosophila mela...  CG10924
4  Glycolysis / Gluconeogenesis - Drosophila me

In [ ]:
from collections import defaultdict

sc.tl.rank_genes_groups(
    oviductless,
    groupby='annotation',  
    method='t-test',       
    key_added='t-test'    
)

clusters = oviductless.uns['t-test']['names'].dtype.names
t_stat_matrix = defaultdict(dict)

for cluster in clusters:
    df = sc.get.rank_genes_groups_df(oviductless, group=cluster, key='t-test')
    for _, row in df.iterrows():
        t_stat_matrix[row['names']][cluster] = row['scores']


t_stats = pd.DataFrame.from_dict(t_stat_matrix, orient='index').fillna(0).T

In [ ]:
import decoupler as dc

scores, norm, pvals = dc.run_gsea(
    mat=t_stats,
    net=net_filtered,
    source='source',
    target='target',
    seed=17,
    use_raw=False,
    verbose=True
)
gsea_results = pd.concat({
    'score': scores.T,
    'norm': norm.T,
    'pval': pvals.T
}, axis=1)  


654 features of mat are empty, they will be removed.
Running gsea on mat with 14 samples and 11806 targets for 53 sources.


100%|██████████| 14/14 [00:15<00:00,  1.09s/it]


In [ ]:
gseares_copy = gsea_results.copy()
gseares_copy.index = gseares_copy.index.str.replace(" - Drosophila melanogaster \(fruit fly\)", "", regex=True)\
                                       .str.replace("/", "_")\
                                       .str.replace(" ", "_")
signif_mask = (gseares_copy['pval'] < 0.1)  # relaxed significance threshold

any_signif = signif_mask.any(axis=1)

all_significant_pathways = gseares_copy.index[any_signif]



heatmap_data = gseares_copy['norm'].loc[all_significant_pathways]
pvals_for_heatmap = gseares_copy['pval'].loc[heatmap_data.index, heatmap_data.columns]

def starify(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    else:
        return ''

asterisk_mask = pvals_for_heatmap.applymap(starify)

g = sns.clustermap(
    heatmap_data,
    cmap="RdBu_r",
    center=0,
    figsize=(14, len(heatmap_data) * 0.4),
    col_cluster=True,
    row_cluster=False,  
    annot=False  
)

g.cax.set_title("NES", fontsize=10)
reordered_rows = heatmap_data.index
reordered_cols = heatmap_data.columns[g.dendrogram_col.reordered_ind]
reordered_mask = asterisk_mask.loc[reordered_rows, reordered_cols]


for i, pathway in enumerate(reordered_rows):
    for j, cluster in enumerate(reordered_cols):
        val = reordered_mask.loc[pathway, cluster]
        if val:
            g.ax_heatmap.text(j + 0.5, i + 0.5, val,
                              ha='center', va='center', color='white', fontsize=9, fontweight='bold')


g.ax_heatmap.set_xlabel("Cluster")
g.ax_heatmap.set_ylabel("Pathway")
g.fig.suptitle("GSEA Pathways (Clustered) with Significance\n(* p<0.05, ** p<0.01, *** p<0.001)", y=1.05)

plt.show()


In [ ]:
sc.pl.umap(
    oviductless, 
    color="annotation",
    ncols=1
)

In [ ]:
dc.run_aucell(
    mat=oviductless,
    net=net_filtered,
    source='source',
    target='target',
    use_raw=False
)

In [ ]:
all_pathways = list(oviductless.obsm["aucell_estimate"].columns)

simplified_names = [p.replace(" - Drosophila melanogaster (fruit fly)", "")
                      .replace("/", "_")
                      .replace(" ", "_") for p in all_pathways]

for original, new_name in zip(all_pathways, simplified_names):
    oviductless.obs[new_name] = oviductless.obsm["aucell_estimate"][original]

for pathway in simplified_names:
    sc.pl.umap(oviductless, color=pathway, cmap="viridis", show=True)


In [71]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Extract wmean pathway scores
pathway_scores = pd.DataFrame(fca_soma.obsm["aucell_estimate"], index=fca_soma.obs_names)

# Add cell labels
pathway_scores["label"] = fca_soma.obs["label"]

pathway_scores.columns = [
    col.replace(" - Drosophila melanogaster (fruit fly)", "").replace("/", "_").replace(" ", "_")
    for col in pathway_scores.columns
]

# Compute mean pathway scores per label (cluster)
pathway_scores_mean = pathway_scores.groupby("label").mean().T




C:\Users\unite\AppData\Local\Temp\ipykernel_7160\2333126459.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pathway_scores_mean = pathway_scores.groupby("label").mean().T
